<a href="https://colab.research.google.com/github/HarshaVardhanLanka/OpenDeepResearchAgent/blob/main/Day_8(Sample).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q -U langgraph langchain langchain-google-genai tavily-python

In [ ]:
import os
from google.colab import userdata
from typing import TypedDict, Annotated, List
from langgraph.graph import StateGraph, END
# LangChain Imports
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate
from tavily import TavilyClient
# --- CONFIGURATION ---
try:
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    TAVILY_API_KEY = userdata.get('TAVILY')
except:
    print("⚠️ Error: Please set GOOGLE_API_KEY and TAVILY in Colab Secrets.")
# Initialize LLM (Gemini Pro)
llm = ChatGoogleGenerativeAI(
    model="gemini-pro-latest", # Flash is faster/cheaper for agent loops
    google_api_key=GOOGLE_API_KEY,
    temperature=0
)
# Initialize Tavily
tavily = TavilyClient(api_key=TAVILY_API_KEY)
print("✅ Setup Complete")

✅ Setup Complete


In [ ]:
class AgentState(TypedDict):
    topic: str              # The user's original input
    research_plan: List[str] # The Planner's list of search queries
    search_results: str     # The raw info gathered by the Searcher
    final_report: str       # The Writer's output

In [ ]:
# --- PLANNER AGENT ---
def planner_node(state: AgentState):
    topic = state["topic"]
    print(f"Thinking: Planner is analyzing '{topic}'...")

    planner_prompt = f"""
    You are a Research Planner.
    Your task is to generate 3 distinct search queries to gather comprehensive information about: "{topic}".
    Return ONLY the 3 queries, separated by newlines. Do not number them.
    """

    response = llm.invoke(planner_prompt)

    # Parse the result into a list
    plan = [line.strip() for line in response.content.split('\n') if line.strip()]

    print(f"   -> Plan generated: {plan}")
    return {"research_plan": plan}

In [ ]:
# --- SEARCHER AGENT ---
def searcher_node(state: AgentState):
    plan = state["research_plan"]
    print(f"Thinking: Searcher is executing {len(plan)} queries...")

    compiled_results = []

    for query in plan:
        try:
            print(f"   -> Searching: {query}")
            # max_results=1 to keep context tight
            results = tavily.search(query=query, max_results=1, search_depth="advanced")

            # Extract content
            if results.get('results'):
                content = results['results'][0]['content']
                compiled_results.append(f"Query: {query}\nResult: {content}\n---")
            else:
                compiled_results.append(f"Query: {query}\nResult: No meaningful data found.\n---")

        except Exception as e:
            print(f"   -> Error searching '{query}': {e}")

    # Join all results into a single string
    search_data_string = "\n".join(compiled_results)

    return {"search_results": search_data_string}

In [ ]:
# --- WRITER AGENT ---
def writer_node(state: AgentState):
    topic = state["topic"]
    data = state["search_results"]
    print("Thinking: Writer is writing the final report...")

    writer_prompt = f"""
    You are a technical writer. Write a professional report on the topic: "{topic}".

    Use the following search data to validate your report:
    {data}

    Requirements:
    1. Use Markdown formatting (Headers, bullet points).
    2. Synthesize the information; do not just list the search results.
    3. Include a "Conclusion" section.
    """

    response = llm.invoke(writer_prompt)

    return {"final_report": response.content}

In [ ]:
# 1. Initialize the Graph
workflow = StateGraph(AgentState)
# 2. Add Nodes (The Agents)
workflow.add_node("planner", planner_node)
workflow.add_node("searcher", searcher_node)
workflow.add_node("writer", writer_node)
# 3. Define Edges (The Workflow Logic)
# START -> Planner -> Searcher -> Writer -> END
workflow.set_entry_point("planner")
workflow.add_edge("planner", "searcher")
workflow.add_edge("searcher", "writer")
workflow.add_edge("writer", END)
# 4. Compile the Graph
app = workflow.compile()
print("✅ Graph Compiled")

✅ Graph Compiled


In [ ]:
from IPython.display import display, Markdown
def run_research_agent(user_topic):
    print(f"🚀 STARTING RESEARCH ON: {user_topic}\n" + "="*40)

    # Initial State
    inputs = {"topic": user_topic}

    # Run the graph
    result = app.invoke(inputs)

    # Output
    print("\n" + "="*40 + "\n✅ RESEARCH COMPLETE")
    display(Markdown(result["final_report"]))
# --- TEST IT ---
topic = input("Enter a topic to research: ")
run_research_agent(topic)

Enter a topic to research: Ocular Toxoplasmosis
🚀 STARTING RESEARCH ON: Ocular Toxoplasmosis
Thinking: Planner is analyzing 'Ocular Toxoplasmosis'...
   -> Plan generated: ['Ocular toxoplasmosis pathophysiology, symptoms, and risk factors', 'Diagnosis and treatment guidelines for ocular toxoplasmosis', 'Recent advances and long-term complications of ocular toxoplasmosis']
Thinking: Searcher is executing 3 queries...
   -> Searching: Ocular toxoplasmosis pathophysiology, symptoms, and risk factors
   -> Searching: Diagnosis and treatment guidelines for ocular toxoplasmosis
   -> Searching: Recent advances and long-term complications of ocular toxoplasmosis
Thinking: Writer is writing the final report...

✅ RESEARCH COMPLETE


# Ocular Toxoplasmosis: A Technical Report

## 1.0 Introduction

Ocular Toxoplasmosis (OT) is an infectious disease of the eye caused by the parasite *Toxoplasma gondii*. It is a leading cause of posterior uveitis (inflammation of the back of the eye) and can lead to significant and permanent vision loss. This report synthesizes current data on the pathophysiology, risk factors, treatment guidelines, and long-term prognosis associated with Ocular Toxoplasmosis.

---

## 2.0 Pathophysiology and Risk Factors

The clinical presentation of OT typically involves retinochoroiditis, an inflammation of the retina and the choroid. Active infections can lead to the formation of toxoplasmic scars, which are associated with a high likelihood of future recurrences.

### 2.1 Risk Factors

The primary risk factor for toxoplasmosis infection is exposure to environments where the *T. gondii* parasite is present. Key risk factors include:

*   **Environmental and Animal Exposure:**
    *   Contact with environments frequented by felines, which are the definitive hosts for the parasite.
    *   Owning more than three cats or kittens.
*   **Dietary Habits:**
    *   Consumption of raw or undercooked meat, including lamb, ground beef, game, and shellfish.
*   **Demographics:**
    *   Studies have identified male sex as a potential risk factor.

---

## 3.0 Treatment and Management

The management of Ocular Toxoplasmosis aims to control the active infection, reduce inflammation, and minimize retinal damage. Treatment strategies vary based on the clinical scenario, such as in cases of congenital infection, pregnancy, or active retinochoroiditis in adults.

### 3.1 Therapeutic Regimens

Several therapeutic options are available for treating active OT:

*   **Preferred First-Line Therapy:** Oral Trimethoprim-Sulfamethoxazole (TMP-SMX) is considered the preferred initial treatment for active disease. It is also effective as a secondary preventive therapy to reduce recurrences.
*   **Classic Triple Therapy:** A combination of oral pyrimethamine, sulfadiazine, and folinic acid (to prevent bone marrow suppression from pyrimethamine).
*   **Intravitreal Therapy:** Injections of clindamycin and dexamethasone directly into the vitreous of the eye serve as an alternative. This approach is particularly useful for patients who are unresponsive to or cannot tolerate oral therapy, or for use during pregnancy.

### 3.2 The Role of Corticosteroids

Corticosteroids are often used in conjunction with antiparasitic medication to control the inflammatory response associated with OT. However, their use requires a critical precaution:

*   **Warning:** Corticosteroids must **not** be administered without a concurrent "shield" of antiparasitic drugs. Unopposed corticosteroid therapy can lead to a severe and rapidly progressing condition known as fulminant necrotizing retinochoroiditis, resulting in catastrophic vision loss.

---

## 4.0 Long-Term Complications and Prognosis

Despite available treatments for active episodes, the long-term prognosis for patients with Ocular Toxoplasmosis can be guarded due to high rates of recurrence and the potential for cumulative ocular damage.

### 4.1 Recurrence Rate

Recurrence is a hallmark of OT. Long-term follow-up studies show that:

*   Up to **79%** of patients experience at least one recurrence.
*   Recurrences predominantly occur in eyes that already have pre-existing toxoplasmic scars.

### 4.2 Visual Outcome

The cumulative damage from initial and recurrent infections poses a significant threat to vision.

*   **Legal Blindness:** Approximately **24%** of patients with OT develop legal blindness in at least one eye.
*   **Congenital OT:** Patients with congenital toxoplasmosis are at a higher risk of developing blindness in both eyes.

### 4.3 Efficacy of Short-Term Treatment

Standard short-term therapeutic modalities used to treat active episodes have not been shown to affect long-term outcomes. Specifically, these treatments do not appear to reduce the rate of future recurrences or improve the final visual outcome. The primary exception is the negative outcome associated with the improper use of corticosteroids without antiparasitic coverage.

---

## 5.0 Conclusion

Ocular Toxoplasmosis is a serious parasitic infection characterized by a high rate of recurrence and a significant risk of permanent vision loss. Primary risk factors are linked to environmental exposure, particularly contact with felines and the consumption of undercooked meat.

Current management focuses on controlling active retinochoroiditis with antiparasitic agents like TMP-SMX or classic triple therapy. While corticosteroids are valuable for managing inflammation, their use is strictly contraindicated without concurrent antiparasitic treatment. Although these therapies are effective for managing acute episodes, they do not prevent future recurrences or alter the long-term visual prognosis. The high incidence of recurrence and potential for legal blindness underscore the chronic and sight-threatening nature of this disease.